# 데이터 가져오기

## 스키마 확인

In [0]:
bucket_name = 's3-lge-he-inbound-kic-prd'
prefix_name = 'adhoc/hedataplfm-878/' # 작업 완료 후, 삭제
parquet_files = [
    '0000_part_00.parquet', 
    '0001_part_00.parquet', 
    '0002_part_00.parquet', 
    '0003_part_00.parquet', 
    '0004_part_00.parquet', 
    '0005_part_00.parquet', 
    '0006_part_00.parquet', 
    '0007_part_00.parquet', 
    '0008_part_00.parquet', 
    '0009_part_00.parquet'
]

for parquet_file in parquet_files:
    df = spark.read.parquet(f"s3a://{bucket_name}/{prefix_name}/{parquet_file}")
    print(parquet_file, len(df.columns))

## 데이터 읽기

In [0]:
df = spark.read.parquet(f"s3a://{bucket_name}/{prefix_name}/")

## 샘플 데이터 확인

In [0]:
display(df.limit(10))

# 암호화

In [0]:
from pyspark.sql.functions import sha2, expr, current_timestamp, col, when, concat, lit, date_format, split, size, substring, to_timestamp

# key
tv_salt = dbutils.secrets.get("admin", "salt")

# hash columns
df = df.withColumn("mac", 
            when(col("mac").isNull() | (col("mac") == ''), col("mac"))\
            .otherwise(sha2(concat(col("mac"), lit(tv_salt)), 256)))
df = df.withColumn("user_number", 
            when(col("user_number").isNull() | (col("user_number") == ''), col("user_number"))\
            .otherwise(sha2(concat(col("user_number"), lit(tv_salt)), 256)))

# 테이블 저장

In [0]:
catalog_name = 'sandbox'
schema_name = 'z_yookyoung_choi'
table_name = 'ucm_log_webos23_kr'

df.write.mode("overwrite").saveAsTable(f"{catalog_name}.{schema_name}.{table_name}")

## 테이블 확인

In [0]:
%python 
spark.sql(f"""
    select *
    from   {catalog_name}.{schema_name}.{table_name}
    limit  100
""").display()